In [1]:
# Import to be able to import python package from src
import sys
sys.path.insert(0, '../src')

In [2]:
import ontime as on
import pandas as pd

# Getting Started

The library is divided in three parts : 

1. `core` for all basic features
2. `module` for all features using core features. E.g. benchmarking, ml preprocessing, etc.
3. `context` for all features related to the usage of onTime in an applied scenario

## `core` Features

This is a low level API. Most objects and functions are accessible in the base object : 
    
    ontime
    ├── detectors
    ├── generators
    ├── Model
    ├── plots
    ├── processors
    └── TimeSeries

For instance : 

In [3]:
ts = on.generators.random_walk().generate(start=pd.Timestamp('01-01-2023'), end=pd.Timestamp('12-31-2023'))
ts[0:5]

<TimeSeries (DataArray) (time: 5, component: 1, sample: 1)> Size: 40B
array([[[ 0.8042713 ]],

       [[ 1.58685749]],

       [[ 0.88496745]],

       [[ 1.07890811]],

       [[-0.41554812]]])
Coordinates:
  * time       (time) datetime64[ns] 40B 2023-01-01 2023-01-02 ... 2023-01-05
  * component  (component) object 8B 'random_walk'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [4]:
ts.plot()

alt.LayerChart(...)

## `module` and `context` features

High level API with various features. Let's load some data for an example :

In [5]:
from darts.datasets import EnergyDataset
ts = EnergyDataset().load()

In [6]:
df = ts.pd_dataframe()
df = df.interpolate()
cols = ['generation biomass', 'generation solar', 'generation nuclear']
df = df[cols]

In [7]:
ts = on.TimeSeries.from_dataframe(df)

In [8]:
ts_uni = ts['generation solar'].slice(pd.Timestamp('2015'), pd.Timestamp('2016'))
ts_multi = ts.slice(pd.Timestamp('2015'), pd.Timestamp('2016'))

### `module` Features

High level API with features related to data processing, ML/AI, etc.

In [9]:
train, test = on.module.processing.common.train_test_split(ts_uni, test_split=0.3)

### `context` Features

High level API with features related to a physical machine or process.

#### Profiler

In [10]:
profiler = on.context.common.Profiler()

What does the common week looks like ?

In [11]:
week_mean = profiler.profile(ts_uni, profiler.Period.WEEKLY, profiler.Aggregation.MEAN).rename({"value": "week_mean"})
week_median = profiler.profile(ts_uni, profiler.Period.WEEKLY, profiler.Aggregation.MEDIAN).rename({"value": "week_median"}) # variable is renamed cause error raises when set to "value" 

In [12]:
(
    on.Plot()
    .add(on.marks.line, week_mean)
    .add(on.marks.line, week_median)
    .show()
)

alt.LayerChart(...)

#### Generic Predictor

In [13]:
model = on.context.common.GenericPredictor()

In [14]:
model.fit(train)

What does the future looks like ?

In [15]:
pred = model.predict(48)

In [16]:
(
    on.Plot()
    .add(on.marks.line, train[-96:].rename({"generation solar": "Training set"}))
    .add(on.marks.line, pred.rename({"generation solar": "Prediction"}))
    .add(on.marks.line, test[:48].rename({"generation solar": "Truth"}), type="dashed")
    .properties(width=600, height=300)
    .show()
)

alt.LayerChart(...)

## Generic Detector

In [17]:
model = on.context.common.GenericDetector()

In [18]:
model.fit(train)

Does the current signal has problem ? 

In [19]:
detected_test = model.detect(test)

In [20]:
(
    on.Plot(test[:72])
    .add(on.marks.line)
    .add(on.marks.mark, data=detected_test[:72].rename({"generation solar": "Anomalies"}), type="dot")
    .properties(width=600, height=300)
    .show()
)

alt.LayerChart(...)

What if we want to have an idea about the future problems ?

In [21]:
predetected = model.predetect(72)

In [22]:
(
    on.Plot(test[:72])
    .add(on.marks.line)
    .add(on.marks.mark, data=predetected[:72].rename({"generation solar": "Anomalies"}), type="dot")
    .properties(width=600, height=300)
    .show()
)

alt.LayerChart(...)